In [4]:
import os
import torch
from PIL import Image, ImageDraw
from torchvision import models, transforms
import torch.nn as nn
from annoy import AnnoyIndex

images_folder = "/Users/younes/Downloads/archive/PetImages/Dog"
weights = models.resnet50(pretrained=True)
model = models.resnet50(weights=weights)
model.fc = nn.Identity()
model.eval()
images = [img for img in os.listdir(images_folder) if img.endswith(('.jpg', '.jpeg', '.png'))]  # Filter only image files

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

annoy_index = AnnoyIndex(2048, 'angular')
annoy_index.load('/Users/younes/ImageSimilarity/dog_index_moreacc.ann')

# Load the input image for which you want to find neighbors
input_image_path = "National-Dog-Day--960x640.jpeg"
input_image = Image.open(input_image_path)
input_tensor = transform(input_image).unsqueeze(0)

if input_tensor.size()[1] == 3:
    output_tensor = model(input_tensor)
    # Find the 7 nearest neighbors of the input image with their distances
    nns, distances = annoy_index.get_nns_by_vector(output_tensor[0], 24, include_distances=True)
    # Create an image grid to display the input image and its 7 nearest neighbors
    image_grid = Image.new('RGB', (1400, 400))
    # Paste the input image in the image grid
    input_image_resized = input_image.resize((200, 200))
    input_image_draw = ImageDraw.Draw(input_image_resized)
    input_image_draw.rectangle([(0, 0), (199, 199)], outline='red', width=8)
    image_grid.paste(input_image_resized)

    # Paste the 7 nearest neighbor images in the image grid
    for j, nn_index in enumerate(nns):
        neighbor_image = Image.open(os.path.join(images_folder, images[nn_index]))
        neighbor_image_resized = neighbor_image.resize((200, 200))
        image_grid.paste(neighbor_image_resized, (200 * (j + 1), 0))
    image_grid.save('/Users/younes/ImageSimilarity/testFolder/nghImage2.png')
    print("Done")


Done
